In [272]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12
import numpy as np
import galois   

In [273]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [348]:
# This script demonstrates the polynomial evaluations with powers of tau
# we see that evaluating a polyomial at tau is the same as computing the inner product of the polynomial coefficients and the powers of tau

tau_testing = GF(20)

# (4x^3 + 2x^2 + 3x + 1) * (x+2)
polynomial = galois.Poly([4,2,3,1], field=GF) * galois.Poly([1,2], field=GF)
print(f"Polynomial: {polynomial}")  # Outputs: 4x^4 + 10x^3 + 7x^2 + 7x + 2

# Evaluate the polynomial at tau.
p_eval_at_tau = polynomial(tau_testing)
print(f"Polynomial evaluated at tau: {p_eval_at_tau}") # Outputs: 3962
p_eval_at_tau_G1 = multiply(G1, int(p_eval_at_tau))

def powers_of_tau_test(tau, degree):
    powers_of_tau_testing = []
    for i in range(degree+1):
        print(f"i = {i}, tau**i = {tau**i}")
        powers_of_tau_testing.append(multiply(G1, int(tau**i)))
    return powers_of_tau_testing

def compute_powers_of_tau_test(tau, degree):
    """Computes and prints the powers of tau up to the specified degree and returns them
    as elliptic curve points by multiplying each power by the base point G1."""
    powers_of_tau = []
    for i in range(degree + 1):
        tau_power = tau ** i
        tau_power_G1 = multiply(G1, int(tau_power))
        powers_of_tau.append(tau_power_G1)
        print(f"i = {i}, tau**i = {tau_power}, tau_power_G1 = {tau_power_G1}")
    return powers_of_tau


# create [τ^0 G1],[τ^1 G1][τ^2 G1]...[τ^degree +1 G1]
powers_of_tau_testing = compute_powers_of_tau_test(tau_testing, polynomial.degree)

# reverse the polynomial coefficients for 4x^4 + 10x^3 + 7x^2 + 7x + 2  to [2, 7, 7, 10, 4]
reversed_polynomial_coeffs = polynomial.coeffs[::-1]
print(f"Reversed polynomial coefficients: {reversed_polynomial_coeffs}")

def inner_product_test(powers_of_tau, coeffs):
    """Computes the elliptic curve point representing the inner product of the reversed polynomial coefficients
    and the powers of tau points."""
    accumulator = Z1
    for tau_power, coeff in zip(powers_of_tau, coeffs):
        term = multiply(tau_power, int(coeff))
        accumulator = add(accumulator, term)
    return accumulator

assert inner_product_test(powers_of_tau_testing, reversed_polynomial_coeffs) == p_eval_at_tau_G1, "doesn't match"

Polynomial: 4x^4 + 10x^3 + 7x^2 + 7x + 2
Polynomial evaluated at tau: 722942
i = 0, tau**i = 1, tau_power_G1 = (1, 2)
i = 1, tau**i = 20, tau_power_G1 = (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992)
i = 2, tau**i = 400, tau_power_G1 = (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714)
i = 3, tau**i = 8000, tau_power_G1 = (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677)
i = 4, tau**i = 160000, tau_power_G1 = (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)
Reversed polynomial coefficients: [ 2  7  7 10  4]


In [295]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [GF(curve_order-2),1,0,3,0,GF(curve_order-1),0,GF(curve_order-2)] # 113 - 2 = 111
    ]))

# in R1CS # m = number of columns # n = number of rows
r1cs_m = L.shape[1]
r1cs_n = L.shape[0]
print(f"m={r1cs_m}, n={r1cs_n}")

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
print(f"witness= {type(witness)}")
print(f"generated L, R, O R1CS matrices and witness vector")
print(f"verifying that L.w * R.w == O.w")
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

m=8, n=5
witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [347]:
# R1CS => QAP
# in QAP m = number of polynomials. n = degree of target polynomial
tau = GF(9)
target_coeffs = GF(np.array([1,2,3,4,5]))
target_polynomial = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_poly_x = matrix_to_vector_of_polynomials(L)
V_vector_of_poly_x = matrix_to_vector_of_polynomials(R)
W_vector_of_poly_x = matrix_to_vector_of_polynomials(O)

qap_m = len(U_vector_of_poly_x)
qap_n = target_polynomial.degree # not sure if this is right
print(f"qap_m={qap_m}, qap_n={qap_n}")
print(f"Witness = {witness}")

# these 2 functions demonstrate that we can either
# 1) multiply polynomials and witness together to 1 polynomial, then evaluate at tau
# 2) evaluate each polynomial at tau, then multiply by witness
def aggregate_vector_of_polynomials_with_witness(vector_of_poly, witness, verbose=False):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        if verbose:
            print(f"scaling witness {witness[i]} by {vector_of_poly[i]}")
        sum += vector_of_poly[i]*witness[i]
    return sum

def aggregate_vector_of_polynomials_with_witness_at_tau(vector_of_poly, witness, verbose=False):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        if verbose:
            print(f"scaling witness {witness[i]} by {vector_of_poly[i]}")
        print(f"vector_of_poly[i] = {vector_of_poly[i]}")
        print(f"vector_of_poly[i](tau) = {vector_of_poly[i](tau)}")
        print(f"witness[i] = {witness[i]}")
        sum += vector_of_poly[i](tau) * witness[i]
    return sum

# This is a polynomial of the inner product of Matrix and Witness
Ua = aggregate_vector_of_polynomials_with_witness(U_vector_of_poly_x, witness, verbose=True)
Va = aggregate_vector_of_polynomials_with_witness(V_vector_of_poly_x, witness)
Wa = aggregate_vector_of_polynomials_with_witness(W_vector_of_poly_x, witness)


print(f"aggregated = {Ua}")
print(f"aggregated = {Va}")
print(f"aggregated = {Wa}")

# We can now create h(x)t(x)
# we test if the polynomial is the same
hx, remainder = divmod(((Ua*Va) - Wa), target_polynomial)
assert remainder == 0, "remainder is not 0"
Ua * Va == Wa + hx * target_polynomial 
print(f"lhs = rhs so they are the same")

# then test if the polynomial evaluated at tau is the same (schwartz zippel)
tau = GF(9)
Ua(tau) * Va(tau) == Wa(tau) + hx(tau) * target_polynomial(tau)
print(f"when evaluated at random number tau, they are the same")

qap_m=8, qap_n=5
Witness = [   1   97    5   10   25  125  100 1250]
scaling witness 1 by 0
scaling witness 97 by 0
scaling witness 5 by 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x
scaling witness 10 by 16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10
scaling witness 25 by 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 18240202393199396018538671454381062573790303667013361953081836822146507079683x^3 + 18240202393199396018538671454381062573790303667013361953081836822146507

In [371]:
# We use trusted setup to hide polynomial coeffs of U, V, W, h, t
from functools import reduce
tau = GF(20)

powers_of_tau_G1 = [multiply(G1, int(tau) ** i) for i in range(0,qap_n)]
powers_of_tau_G2 = [multiply(G2, int(tau) ** i) for i in range(0,qap_n)]
# target_powers_of_tau_G1 = [multiply(G1, int(tau) ** i * int(t_of_tau)) for i in range(0,target_polynomial.degree)]
target_powers_of_tau_G1 = [multiply(G1, int(tau) ** i) for i in range(0,target_polynomial.degree)]
t_of_tau = target_polynomial(tau)
target_powers_of_tau_G1_eval_at_tau = [multiply(target_powers_of_tau_G1[i], int(t_of_tau)) for i in range(0, len(target_powers_of_tau_G1))]
print("len",len(powers_of_tau_G1))

# Quick sanity test - this is not encrypted and wouldn't happen in real life
Ua_tau = Ua(tau)
Va_tau = Va(tau)
Wa_tau = Wa(tau)
hx_tau = hx(tau)
tx_tau = target_polynomial(tau)
# print(f"Ua(tau) = {Ua_tau}")
# print(f"Va(tau) = {Va_tau}")
# print(f"Wa(tau) = {Wa_tau}")
# print(f"hx(tau) = {hx_tau}")
# print(f"tx(tau) = {tx_tau}")
# print(f"ht= {hx_tau * tx_tau}")

assert Ua_tau * Va_tau == Wa_tau + hx_tau * tx_tau, "Ua_tau * Va_tau != Wa_tau + hx_tau * tx_tau"

def inner_product(powers_of_tau, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau, coeffs)), Z1)

def inner_product_iterable(powers_of_tau, coeffs):
    # print(len(powers_of_tau_G1), " eq? ", len(coeffs))
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        # print(f"Iteration: {i}, Coeff: {coeff}, Point: {point}")
        result = add(result, multiply(point, int(coeff)))
    return result
 

Ua_tau_G1 = inner_product_iterable(powers_of_tau_G1, Ua.coeffs[::-1])
Va_tau_G2 = inner_product_iterable(powers_of_tau_G2, Va.coeffs[::-1])
Wa_tau_G1 = inner_product_iterable(powers_of_tau_G1, Wa.coeffs[::-1])

print("Ua tau g1",Ua_tau_G1)

# we use from [o:hx.degree + 1] because we need to cover all hx coeffs
ht_tau_G1 = inner_product_iterable(target_powers_of_tau_G1_eval_at_tau[0:hx.degree + 1], hx.coeffs[::-1])

print("Ua tau g1",Ua_tau_G1)
print("Va tau g1",Va_tau_G2)
print("Wa tau g1",Wa_tau_G1)
print("httau",ht_tau_G1)

AB = pairing(Va_tau_G2, neg(Ua_tau_G1))
CD = pairing(G2, add(Wa_tau_G1, ht_tau_G1))
final_exponentiate(AB * CD) == FQ12.one()

len 5
Ua tau g1 (10761961764129973982054763719362943500866170651873734401430882177283055329417, 5058895679614798810943118395621508047500638844754741455340774317600459389282)
Ua tau g1 (10761961764129973982054763719362943500866170651873734401430882177283055329417, 5058895679614798810943118395621508047500638844754741455340774317600459389282)
Va tau g1 ((14676596619546708998128589777444655676977278549271023549538909836586459894341, 11919835763118668667740900971595365782274213478929042926599835422246695122845), (4181410176679702185692223732593891253384989642736027109804322446705565922423, 8269324330636396465404020993058672731247265150570478427587813989536384471923))
Wa tau g1 (11006377201644525345426928991404003103659940521647545481641368342770471795148, 5625735634426074623513914315028312426791663040205825753846553583577855352457)
httau (7590285867717173171246348344240436388940237951733020390870567473915245420644, 1639484974281784856678405376279179323802291637797507447031121243169689759092

True

In [372]:
# shift lhs and rhs by alpha and beta
# Trusted Setup
alpha = GF(2)
beta = GF(3)
AlphaG1 = multiply(G1, int(alpha))
BetaG2 = multiply(G2, int(beta))

print(hx)

# # reminder
# # qap_m = len(U_vector_of_poly_x)
# # qap_n = target_polynomial.degree # not sure if this is right

# encrypted evaluation: shifts the U and V polynomial by alpha and beta, evaluates at powers of tau
# Trusted Setup completes this with unencrypted tau, alpha, beta values
powers_of_tau_C_G1 = []
for i in range(0, qap_m):
    iterator_tau = tau
    # print(f"polynomial ui = {U_vector_of_poly_x[i]}")
    # print(f"iterator = {i}, tau = {tau}, tau**i = {tau}, U_vector_of_poly_x[i](tau**i) = {U_vector_of_poly_x[i](iterator_tau)}")
    iterator_u = U_vector_of_poly_x[i](iterator_tau)
    iterator_v = V_vector_of_poly_x[i](iterator_tau)
    iterator_w = W_vector_of_poly_x[i](iterator_tau)
    sum = int(beta * iterator_u + alpha * iterator_v + iterator_w) * int(witness[i])
    print(f"sum polynomial for iterator {i} = {sum}")
    powers_of_tau_C_G1.append(multiply(G1, sum))
    # print(f"current powers of tau G1 = {powers_of_tau_C_G1}")
print(f"powers_of_tau_C_G1 = {powers_of_tau_C_G1}")
print(f"length: powers_of_tau_C_G1 = {len(powers_of_tau_C_G1)}")

def inner_product_iterable_test(powers_of_tau, coeffs):
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        result = add(result, multiply(point, int(coeff)))
    return result

def powers_of_tau_innerproduct_poly_i(powers_of_tau, vector_of_polys, witness):
    accumulator = Z1
    for i in range(0, len(vector_of_polys)):
        iterator_point = inner_product_iterable_test(powers_of_tau, vector_of_polys[i].coeffs[::-1])
        accumulator = add(accumulator, multiply(iterator_point, int(witness[i])))
    return accumulator    

A_11 = powers_of_tau_innerproduct_poly_i(powers_of_tau_G1, U_vector_of_poly_x, witness)
B_22 = powers_of_tau_innerproduct_poly_i(powers_of_tau_G2, V_vector_of_poly_x, witness)
A_11_new = add(A_11, AlphaG1)
B_22_new = add(B_22, BetaG2)

print("A11 new",A_11_new)
print("B22 new",B_22_new)

C_11_new = Z1
for i in range(len(powers_of_tau_C_G1)):
    C_11_new = add(C_11_new, powers_of_tau_C_G1[i])
    
print(f"i = {i}, C_11_new = {C_11_new}")

C_11_new_httau = add(C_11_new, ht_tau_G1)

# hx is the same, ht_tau_G1 is the same
print("httau = ", ht_tau_G1)



AB = pairing(B_22_new, neg(A_11_new))
CD = pairing(G2, C_11_new_httau)
EF = pairing(BetaG2, AlphaG1)
final_exponentiate(AB * CD * EF) == FQ12.one()

1824020239319939601853867145438106257379030366701336195308183682214650707866x^3 + 8360092763549723175163557749924653679653889180714457561829175210150482412190x^2 + 16720185527099446350327115499849307359307778361428915123658350420300964821680x + 14592161914559516814830937163504850059032242933610689562465469457717205664478
sum polynomial for iterator 0 = 21888242871839275222246405745257275088548364400416034343698204186575808487865
sum polynomial for iterator 1 = 375972
sum polynomial for iterator 2 = 109441214359196376111232028726286375442741822002080171718491020932879042134685
sum polynomial for iterator 3 = 851580
sum polynomial for iterator 4 = 547206071795981880556160143631431877213709110010400858592455104664395211497925
sum polynomial for iterator 5 = 2736030358979909402780800718157159386068545550052004292962275523321976054402000
sum polynomial for iterator 6 = 2827200
sum polynomial for iterator 7 = 27360303589799094027808007181571593860685455500520042929622755233219760591662500
po

True

In [279]:
# separate public and private inputs
# public input, computed by verifier
public_witness_length = 2
public_input = []

for i in range(public_witness_length):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    public_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

# we now have 2 points representing Bui, Avi, Wvi. Let's squash the witness in and add to 1 point
public_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(public_input, witness[:public_witness_length])), Z1)

# private input, computed by prover. Prover has access to witness and 
private_input = []
for i in range(public_witness_length,len(witness)):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    private_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

private_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(private_input, witness[public_witness_length:])), Z1)

C_new = add(Cprime, ht_tau)



iteration 0 witness:  1
iteration 1 witness:  97
iteration 2 witness:  5
iteration 3 witness:  10
iteration 4 witness:  25
iteration 5 witness:  125
iteration 6 witness:  100
iteration 7 witness:  1250


In [280]:
# Introducing Gamma, Delta
gamma = GF(4)
delta = GF(5)
gamma_inv = np.reciprocal(gamma)
delta_inv = np.reciprocal(delta)

GammaG2 = multiply(G2, int(gamma))
DeltaG2 = multiply(G2, int(delta))

# powers of tau for public input from i = 0 to i = 1 for the 0, 1 pos of [1,out,x,y,...] 
public_witness_length = 2

# public
public_powers_of_tau_G1 = [multiply(point, int(gamma_inv)) for point in powers_of_tau_C_G1[:public_witness_length]]
gamma_pairing = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(public_powers_of_tau_G1, witness[0:public_witness_length])), Z1)


# private
private_powers_of_tau_G1 = [multiply(point, int(delta_inv)) for point in powers_of_tau_C_G1[public_witness_length+1:len(witness)]]
Cprime_delta = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(private_powers_of_tau_G1, witness[public_witness_length:len(witness)])), Z1)


#ht_tau is from i = 0 -> m. Needs to change to l -> m
delta_ht_tau = multiply(ht_tau, int(delta_inv))
C_prime_new = add(Cprime_delta, ht_tau)
C_new = add(Cprime, ht_tau)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(GammaG2, gamma_pairing)
rhs_3 = pairing(DeltaG2, C_prime_new)
rhs = rhs_1 * rhs_2 * rhs_3

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")

not verified


In [281]:
import numpy as np
from scipy.interpolate import lagrange
x = np.array([1,2,3])
y = np.array([1,1,0])
print(lagrange(x, y))

      2
-0.5 x + 1.5 x
